In [1]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

# Load the model
model = load_model("15_point_plastic_classifier_model3.keras")


In [3]:
# Sample data based on your provided example (modify this as needed)
sample_data = [
    [2493065, 4558145, 7638499, 8387631, 8388397, 8389475, 4054559, 2106185, 3340801, 5935779, 8389399, 8388961, 8386511, 6563979, 3097997]
]

# Create a function to generate a representative dataset
def representative_dataset():
    for data_point in sample_data:
        # Reshape and convert to float32 (modify this if your model's input format is different)
        input_data = tf.constant(data_point, shape=(1, 15), dtype=tf.float32)
        yield [input_data]

# Convert the model to TFLite format with quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_dataset  # Specify the representative dataset

tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('your_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\kylej\AppData\Local\Temp\tmpmbwzk5mq\assets


INFO:tensorflow:Assets written to: C:\Users\kylej\AppData\Local\Temp\tmpmbwzk5mq\assets
c:\Users\kylej\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\lite\python\convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [24]:
data = pd.read_csv("AllTogetherPE.csv")

# Separate features and labels
X = data.drop(columns=["Material_Type"])
y = data["Material_Type"]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets (Normal)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=54)


# Normalize the manual input if needed
X_test_normalized = ((X_test - 0) / (8410000 - 0))
# X_test_normalized = (X_test - X_train.min()) / (X_train.max() - X_train.min())

# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)

# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -5:]
top_labels = top_predictions[:, ::-1]

# print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(len(predicted_labels)):
    num_predictions = min(5, len(np.unique(top_labels[i])))
    if y_test[i] == predicted_labels[i]:
        CorF = "Correct,"
    else:
        CorF = "False,"
    print(f"Sample {i+1}: {CorF} it was {label_encoder.classes_[y_test[i]]}")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][1-j]:.2f}% likelihood")



4/4 [==============================] - 0s 2ms/step
Test Accuracy: 0.9487179517745972
Sample 1: Correct, it was PE
  Prediction 1: PE with 0.01% likelihood
  Prediction 2: Other with 0.00% likelihood
  Prediction 3: PP with 99.94% likelihood
  Prediction 4: Alu with 0.05% likelihood
Sample 2: Correct, it was PE
  Prediction 1: PE with 0.00% likelihood
  Prediction 2: Other with 0.00% likelihood
  Prediction 3: PP with 100.00% likelihood
  Prediction 4: Alu with 0.00% likelihood
Sample 3: Correct, it was PE
  Prediction 1: PE with 0.04% likelihood
  Prediction 2: PP with 0.00% likelihood
  Prediction 3: Other with 99.92% likelihood
  Prediction 4: Alu with 0.04% likelihood
Sample 4: Correct, it was PE
  Prediction 1: PE with 0.00% likelihood
  Prediction 2: Other with 0.00% likelihood
  Prediction 3: PP with 99.82% likelihood
  Prediction 4: Alu with 0.18% likelihood
Sample 5: Correct, it was Alu
  Prediction 1: Alu with 0.00% likelihood
  Prediction 2: Other with 0.00% likelihood
  Pred

In [25]:

exclude_alu_indices = y != 'ALU'
X_filtered = X[exclude_alu_indices]
y_filtered = y_encoded[exclude_alu_indices]
# Split the filtered data into train and test sets (Filtered ALU)
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=50)
#features
X_test_normalized = ((X_test - 0) / (8410000 - 0))

# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)

# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -4:]
top_labels = top_predictions[:, ::-1]

# print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(len(predicted_labels)):
    num_predictions = min(4, len(np.unique(top_labels[i])))
    if y_test[i] == predicted_labels[i]:
        CorF = "Correct,"
    else:
        CorF = "False,"
    print(f"Sample {i+1}: {CorF} it was {label_encoder.classes_[y_test[i]]}")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][3-j]:.2f}% likelihood")


4/4 [==============================] - 0s 4ms/step - loss: 0.1323 - accuracy: 0.9487


4/4 [==============================] - 0s 2ms/step
Test Accuracy: 0.9487179517745972
Sample 1: Correct, it was PE
  Prediction 1: PE with 99.94% likelihood
  Prediction 2: Other with 0.06% likelihood
  Prediction 3: Alu with 0.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 2: Correct, it was Other
  Prediction 1: Other with 99.91% likelihood
  Prediction 2: PE with 0.09% likelihood
  Prediction 3: Alu with 0.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 3: Correct, it was PE
  Prediction 1: PE with 99.85% likelihood
  Prediction 2: PP with 0.07% likelihood
  Prediction 3: Other with 0.07% likelihood
  Prediction 4: Alu with 0.00% likelihood
Sample 4: Correct, it was PE
  Prediction 1: PE with 99.97% likelihood
  Prediction 2: Other with 0.03% likelihood
  Prediction 3: PP with 0.00% likelihood
  Prediction 4: Alu with 0.00% likelihood
Sample 5: Correct, it was Other
  Prediction 1: Other with 99.99% likelihood
  Prediction 2: PE with 0.00% likelihood
  

In [26]:
# Get manual input from the user
manual_input = np.array([[1271389,2107581,3534859,5988057,6233631,3880377,1498541,985865,1561809,2707983,4719147,6111335,5063453,2701889,1262325]])  # Replace with your input values

# Normalize the manual input if needed
normalized_manual_input = ((manual_input - 0) / (8410000 - 0))

# Make predictions on manual input
predictions = model.predict(normalized_manual_input)
print(predictions)

# Get the top prediction
top_prediction = np.argmax(predictions)
print(top_prediction)

top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
print(top_plastic_type)
likelihood = predictions[0][top_prediction] * 100

# Display the prediction for manual input
print(f"Manual Input Prediction:")
print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

1/1 [==============================] - 0s 46ms/step
[[5.3921148e-08 6.4218715e-05 6.2944286e-02 9.3699145e-01]]
3
PP
Manual Input Prediction:
  Prediction: PP with 93.70% likelihood


In [28]:
import numpy as np  # Add this line to import the NumPy library
import serial
from sklearn.preprocessing import LabelEncoder


# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Initialize a variable to store the received data
received_data = ""

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()
        
        # Append the received line to the variable
        received_data += line

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in received_data.split(',')]

        # Convert the list of values into a NumPy array
        manual_input = np.array([data_values])

        # Normalize the manual input if needed
        normalized_manual_input = ((manual_input - 0) / (8410000 - 0))

        # Make predictions on manual input
        predictions = model.predict(normalized_manual_input)
        # print(predictions)

        # Get the top prediction
        top_prediction = np.argmax(predictions)
        print(top_prediction)

        top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
        print(top_plastic_type)
        likelihood = predictions[0][top_prediction] * 100

        # Display the prediction for manual input
        print(f"Manual Input Prediction:")
        print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

        received_data = ""

        # Send the top plastic type back over serial
        ser.write(top_plastic_type.encode('utf-8'))

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the serial port when done
    ser.close()

# Now, you can use the 'received_data' variable for further processing
print("total data:\n", received_data)


1/1 [==============================] - 0s 17ms/step
3
PP
Manual Input Prediction:
  Prediction: PP with 92.09% likelihood
1/1 [==============================] - 0s 34ms/step
3
PP
Manual Input Prediction:
  Prediction: PP with 92.28% likelihood
1/1 [==============================] - 0s 32ms/step
3
PP
Manual Input Prediction:
  Prediction: PP with 90.88% likelihood
1/1 [==============================] - 0s 22ms/step
3
PP
Manual Input Prediction:
  Prediction: PP with 42.01% likelihood
1/1 [==============================] - 0s 22ms/step
3
PP
Manual Input Prediction:
  Prediction: PP with 75.27% likelihood
1/1 [==============================] - 0s 26ms/step
2
PE
Manual Input Prediction:
  Prediction: PE with 99.05% likelihood
1/1 [==============================] - 0s 36ms/step
2
PE
Manual Input Prediction:
  Prediction: PE with 99.84% likelihood
1/1 [==============================] - 0s 9ms/step
2
PE
Manual Input Prediction:
  Prediction: PE with 58.43% likelihood
1/1 [====================

SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))

In [ ]:
import numpy as np
import serial
import csv  # Import the csv module
from sklearn.preprocessing import LabelEncoder

# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Create and open a CSV file for writing
csv_file = open('HDPE2.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in line.split(',')]

        print(data_values)

        # Write the data values to the CSV file as a new row
        csv_writer.writerow(data_values)

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the CSV file and the serial port when done
    csv_file.close()
    ser.close()

# The received data has been saved in the 'received_data.csv' file
print("Data saved to 'PP2.csv'")



[987985.0, 2033589.0, 3299665.0, 5969235.0, 6046363.0, 3364983.0, 1387929.0, 766707.0, 1395625.0, 2550645.0, 4582413.0, 6008585.0, 4738749.0, 2408671.0, 1101923.0]
[992033.0, 2043305.0, 3311875.0, 5976717.0, 6030259.0, 3352487.0, 1385053.0, 766433.0, 1399529.0, 2558603.0, 4597953.0, 6011647.0, 4719877.0, 2393163.0, 1099957.0]
[967793.0, 2617937.0, 3975225.0, 5542917.0, 5185701.0, 2428671.0, 1567971.0, 588693.0, 1670571.0, 3171047.0, 4719881.0, 5366329.0, 3813949.0, 2016815.0, 1102557.0]
[1073025.0, 2596131.0, 4221975.0, 6333217.0, 6243891.0, 3091343.0, 1609077.0, 810399.0, 1699357.0, 3270559.0, 5233773.0, 6297655.0, 4684141.0, 2371953.0, 1231931.0]
[1013065.0, 2494465.0, 4118677.0, 5690299.0, 5030207.0, 2606377.0, 1612049.0, 801739.0, 1627355.0, 3175037.0, 4868199.0, 5396979.0, 3865465.0, 2132271.0, 1225569.0]
[1264449.0, 2186883.0, 3859033.0, 5984245.0, 6720187.0, 3444151.0, 1660649.0, 952065.0, 1597127.0, 2905361.0, 4880265.0, 6354061.0, 5079533.0, 2559809.0, 1325237.0]
[1265169.0, 2

SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))